In [4]:
import pygame, random

# ---------------- CONFIG ----------------
WORLD_WIDTH, WORLD_HEIGHT = 1400, 800
ROAD_Y = WORLD_HEIGHT // 2
LANES = [ROAD_Y - 60, ROAD_Y, ROAD_Y + 60]
CAR_LENGTH = 45
CAR_WIDTH = 22
SAFE_DISTANCE = 140
SENSOR_RANGE = 300
DT = 0.05

# COLORS
COLOR_BG = (25, 28, 33)
COLOR_ROAD = (60, 65, 75)
COLOR_ACCIDENT = (220, 60, 60)
COLOR_CAR = (100, 200, 255)

# ---------------- CLASSES ----------------
class Vehicle:
    def __init__(self, x, y, speed):
        self.x, self.y = x, y
        self.speed = speed
        self.target_speed = speed
        self.color = random.choice([(255, 80, 80), (80, 200, 255), (255, 200, 0), (80, 255, 100)])
        self.lane = y
        self.switching = False
        self.sensor_on = False

    def step(self):
        # Smooth speed change
        if abs(self.speed - self.target_speed) > 1:
            self.speed += (self.target_speed - self.speed) * 0.2
        self.x += self.speed * DT

class Hazard:
    def __init__(self, x, y):
        self.x, self.y = x, y

# ---------------- SIMULATION ----------------
class Simulation:
    def __init__(self):
        self.vehicles = []
        self.hazard = Hazard(WORLD_WIDTH//2, ROAD_Y)
        self.make_traffic()

    def make_traffic(self):
        for i in range(15):
            lane = random.choice(LANES)
            x = random.randint(0, WORLD_WIDTH)
            self.vehicles.append(Vehicle(x, lane, random.uniform(20, 30)))

    def get_front_vehicle(self, v):
        nearest, dist = None, 9999
        for o in self.vehicles:
            if o == v: continue
            if abs(o.y - v.y) < 25 and o.x > v.x:
                d = o.x - v.x
                if d < dist:
                    dist = d
                    nearest = o
        return nearest, dist

    def is_lane_clear(self, y_target, x_pos):
        for o in self.vehicles:
            if abs(o.y - y_target) < 25 and abs(o.x - x_pos) < 100:
                return False
        return True

    def step(self):
        for v in self.vehicles:
            # SENSOR CHECK for accident in same lane
            dist_to_acc = self.hazard.x - v.x
            same_lane = abs(v.y - self.hazard.y) < 30

            v.sensor_on = same_lane and 0 < dist_to_acc < SENSOR_RANGE

            # detect front car
            front, dist_front = self.get_front_vehicle(v)

            if same_lane and 0 < dist_to_acc < SENSOR_RANGE:
                # try to switch lane if possible
                for offset in [-60, 60]:
                    new_lane = v.lane + offset
                    if new_lane in LANES and self.is_lane_clear(new_lane, v.x):
                        v.y = new_lane
                        v.lane = new_lane
                        v.switching = True
                        break
                else:
                    # if cannot switch → stop before accident
                    if dist_to_acc < SAFE_DISTANCE:
                        v.target_speed = 0
                    else:
                        v.target_speed = 15
            else:
                # normal traffic flow
                v.switching = False
                if front and dist_front < SAFE_DISTANCE:
                    v.target_speed = 0
                else:
                    v.target_speed = 25

            v.step()

        # wrap-around
        for v in self.vehicles:
            if v.x > WORLD_WIDTH + 50:
                v.x = -300
                v.speed = random.uniform(20, 30)

# ---------------- VISUAL ----------------
class Visual:
    def __init__(self, sim):
        pygame.init()
        self.sim = sim
        self.screen = pygame.display.set_mode((WORLD_WIDTH, WORLD_HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("Arial", 20)

    def draw(self):
        s = self.screen
        s.fill(COLOR_BG)
        pygame.draw.rect(s, COLOR_ROAD, (0, ROAD_Y - 120, WORLD_WIDTH, 240))

        # Lanes
        for y in LANES:
            for x in range(0, WORLD_WIDTH, 50):
                pygame.draw.line(s, (90, 90, 90), (x, y), (x+25, y), 2)

        # Accident
        h = self.sim.hazard
        pygame.draw.rect(s, COLOR_ACCIDENT, (h.x-25, h.y-15, 50, 30))
        pygame.draw.line(s, (255,255,255), (h.x-20,h.y-10),(h.x+20,h.y+10),3)
        pygame.draw.line(s, (255,255,255), (h.x-20,h.y+10),(h.x+20,h.y-10),3)

        # Cars
        for v in self.sim.vehicles:
            pygame.draw.rect(s, v.color, (v.x-20, v.y-10, 40, 20))
            if v.sensor_on:
                pygame.draw.circle(s, (0,255,255), (int(v.x+18), int(v.y)), 3)
            if v.target_speed == 0:
                pygame.draw.circle(s, (255,0,0), (int(v.x-16), int(v.y-6)), 3)
                pygame.draw.circle(s, (255,0,0), (int(v.x-16), int(v.y+6)), 3)

        text = self.font.render("🚧 Cars avoid or stop near accident (cyan=detecting)", True, (230,230,240))
        s.blit(text, (20,20))
        pygame.display.flip()

    def loop(self):
        run=True
        while run:
            for e in pygame.event.get():
                if e.type==pygame.QUIT:
                    run=False
            self.sim.step()
            self.draw()
            self.clock.tick(30)
        pygame.quit()

# ---------------- RUN ----------------
sim = Simulation()
vis = Visual(sim)
vis.loop()
